In [1]:
%reload_ext autoreload
%autoreload 

In [2]:
import numpy as np
from time import time

In [3]:
import cProfile
import pstats
import allel
import pandas as pd

In [4]:
%aimport malariagen_data.ag3

In [5]:
%aimport malariagen_data.veff 

In [6]:
veff = malariagen_data.veff

In [7]:
ag3 = malariagen_data.Ag3("simplecache::gs://vo_agam_release", simplecache=dict(cache_storage="gcs_cache"))

In [8]:
genome = ag3.open_genome()
genome

<zarr.hierarchy.Group '/'>

In [ ]:
list(genome)

In [ ]:
genome["3R"][0:10].tobytes().decode()

In [9]:
geneset = ag3.geneset()
geneset

,seqid,source,type,start,end,score,strand,phase,ID,Parent,Name
0,2L,VectorBase,chromosome,1,49364325,NaN,NaN,NaN,2L,NaN,NaN
1,2L,VectorBase,gene,157348,186936,NaN,-,NaN,AGAP004677,NaN,NaN
2,2L,VectorBase,mRNA,157348,181305,NaN,-,NaN,AGAP004677-RA,AGAP004677,NaN
3,2L,VectorBase,three_prime_UTR,157348,157495,NaN,-,NaN,NaN,AGAP004677-RA,NaN
4,2L,VectorBase,exon,157348,157623,NaN,-,NaN,NaN,AGAP004677-RA,AGAP004677-RB-E4
...,...,...,...,...,...,...,...,...,...,...,...
196140,Y_unplaced,VectorBase,five_prime_UTR,47932,48111,NaN,+,NaN,NaN,AGAP029375-RA,NaN
196141,Y_unplaced,VectorBase,exon,47932,48138,NaN,+,NaN,NaN,AGAP029375-RA,AGAP029375-RA-E2
196142,Y_unplaced,VectorBase,CDS,48112,48138,NaN,+,0.0,AGAP029375-PA,AGAP029375-RA,NaN
196143,Y_unplaced,VectorBase,exon,48301,48385,NaN,+,NaN,NaN,AGAP029375-RA,AGAP029375-RA-E3


In [ ]:
geneset[geneset.Parent == 'AGAP004679-RB']

In [ ]:
%%time
# this will take some time, loading gff, one-off cost
ann = veff.Annotator(
    genome=genome,
    geneset=geneset,
)

In [ ]:
%%time
for effect in ann.get_effects(chrom='2L', pos=2422652, ref='A', alt='T',
                                  transcript_ids=["AGAP004707-RD"]):
       print(effect)

In [ ]:
for a in effect:
    print(a)

In [ ]:
effect.effect, effect.impact

In [ ]:
effect.ref_codon, effect.alt_codon

In [ ]:
effect.aa_pos, effect.ref_aa, effect.alt_aa, effect.aa_change

In [ ]:
type(effect.aa_pos)

In [ ]:
pos, ref, alt = ag3.snp_sites('2L')

In [ ]:
import allel

In [ ]:
start_idx = allel.SortedIndex(pos).locate_key(2429745)
start_idx

In [ ]:
pp = pos[start_idx:start_idx+100].compute()
rr = ref[start_idx:start_idx+100].compute()
aaa = alt[start_idx:start_idx+100].compute()

In [ ]:
def testf(n, show=False):
    # loop over sites
    for i, (p, r, aa) in enumerate(zip(pp, rr, aaa)):
        if i < n:
            # loop over alt alleles
            for a in aa:
                for effect in ann.get_effects(chrom='2L', pos=p, ref=r.decode(), alt=a.decode(),
                                              transcript_ids=["AGAP004707-RA"]):
                    if show:
                        print(effect)
    

In [ ]:
%%time
testf(20)

In [ ]:
%%time
x = ref.compute()

In [ ]:
x.nbytes

In [ ]:
cProfile.run('testf(100)', sort='cumtime')

In [ ]:
cProfile.run('testf(20)', sort='time')

In [ ]:
ann.get_feature("AGAP004707-RD")

In [ ]:
ann.get_children("AGAP004707")

In [ ]:
ann.find("2L", 2358158, 2358159)

# ag3.snp_effects appears to work for vgsc and some other genes but most genes seem to fail
- the number of effects is less than the number of rows of positions

#### run veff locally and try to debug

In [ ]:
genome = ag3.open_genome()
genome

In [9]:
gste2 = 'AGAP009194-RA'
site_mask = 'gamb_colu'

In [ ]:
geneset[geneset.Parent == gste2]

In [ ]:
%%time
for effect in ann.get_effects(chrom='3R', pos=28598166, ref='A', alt='G',
                                  transcript_ids=[gste2]):
       print(effect)

In [ ]:
feature = ann.get_feature(gste2)
#feature = ann.get_feature('AGAP004707-RA')
contig = feature[0]
start = feature[3]
stop = feature[4]
strand = feature[6]

In [ ]:
feature

In [ ]:
sites = ag3.snp_sites(contig=contig, site_mask=site_mask)

In [ ]:
# sites are dask arrays, turn pos into sorted index
pos = allel.SortedIndex(sites[0].compute())
# locate transcript range
loc = pos.locate_range(start, stop)
# dask compute on the sliced arrays to speed things up
ref = sites[1][loc].compute()
alt = sites[2][loc].compute()

In [ ]:
loc

In [ ]:
df_in = pd.DataFrame()
df_in["position"] = np.asarray(pos[loc])
df_in["ref_allele"] = [q.tobytes().decode() for q in np.asarray(ref)]
# bytes within lists within lists...
df_in["alt_alleles"] = [list(q.tobytes().decode()) for q in list(alt)]

In [ ]:
df_in.shape[0] *3

In [ ]:
#explode
df_effects = df_in.explode("alt_alleles").reset_index(drop=True)

In [ ]:
df_effects.shape

In [ ]:
df_effects

In [ ]:
leffect = []
limpact = []
lref_codon = []
lalt_codon = []
laa_pos = []
lref_aa = []
lalt_aa = []
laa_change = []
lpos = []

pos = []

for row in df_effects.itertuples(index=True):
    pos.append(row.position)
    for effect in ann.get_effects(
        chrom=contig,
        pos=row.position,
        ref=row.ref_allele,
        alt=row.alt_alleles,
        transcript_ids=[gste2],
    ):

        leffect.append(effect.effect)
        lpos.append(effect.ref_start)
        limpact.append(effect.impact)
        lref_codon.append(effect.ref_codon)
        lalt_codon.append(effect.alt_codon)
        laa_pos.append(effect.aa_pos)
        lref_aa.append(effect.ref_aa)
        lalt_aa.append(effect.alt_aa)
        laa_change.append(effect.aa_change)

In [ ]:
len(pos)

In [ ]:
len(lpos)

In [ ]:
s = set(lpos)
missing = [x for x in pos if x not in s]

In [ ]:
len(missing) + len(lpos)

In [ ]:
missing

In [ ]:
%%time
for effect in ann.get_effects(chrom='3R', pos=28597652, ref='G', alt='A',
                                  transcript_ids=[gste2]):
       print(effect)

In [ ]:
e = ag3.snp_single_effect('3R', pos=28598166, ref='A', alt='G', transcript=[gste2])
e

In [ ]:
type(e)

In [ ]:
e = ag3.snp_single_effect('3R', pos=28597652, ref='G', alt='A',
                                  transcript=[gste2])
e

In [ ]:
assert e.effect == 'UTR_VARIANT'

# working out test parameters

### reverse strand

In [ ]:
df = ag3.snp_effects(gste2, site_mask)

In [ ]:
a = df.columns[:]

In [ ]:
b = ['position', 'ref_allele', 'alt_alleles', 'effect', 'impact',
       'ref_codon', 'alt_codon', 'aa_pos', 'ref_aa', 'alt_aa', 'aa_change']

In [ ]:
assert isinstance(df, pd.DataFrame)

In [ ]:
for e in b:
    assert e in a

In [ ]:
assert df.shape == (2838, 11)

In [ ]:
df[df.effect == 'FIVE_PRIME_UTR']

In [ ]:
df.iloc[1451]

In [ ]:
assert df.iloc[1451].aa_change == 'I114T'

In [ ]:
df = ag3.snp_effects(gste2, site_mask)

In [ ]:
df

### forward strand

In [ ]:
gste6 = "AGAP009196-RA"

In [ ]:
df = ag3.snp_effects(gste6, site_mask)

In [ ]:
df.shape

In [ ]:
df[900:]

### introns in 5' UTRs

In [14]:
geneset[geneset.Parent == "AGAP004679-RB"]

,seqid,source,type,start,end,score,strand,phase,ID,Parent,Name
34,2L,VectorBase,exon,207894,207953,NaN,+,NaN,NaN,AGAP004679-RB,AGAP004679-RB-E1
35,2L,VectorBase,five_prime_UTR,207894,207953,NaN,+,NaN,NaN,AGAP004679-RB,NaN
36,2L,VectorBase,five_prime_UTR,208393,208416,NaN,+,NaN,NaN,AGAP004679-RB,NaN
37,2L,VectorBase,exon,208393,208581,NaN,+,NaN,NaN,AGAP004679-RB,AGAP004679-RB-E2
38,2L,VectorBase,CDS,208417,208581,NaN,+,0.0,AGAP004679-PB,AGAP004679-RB,NaN
39,2L,VectorBase,exon,208639,208824,NaN,+,NaN,NaN,AGAP004679-RB,AGAP004679-RB-E3
40,2L,VectorBase,CDS,208639,208824,NaN,+,0.0,AGAP004679-PB,AGAP004679-RB,NaN
41,2L,VectorBase,exon,208897,209394,NaN,+,NaN,NaN,AGAP004679-RB,AGAP004679-RB-E4
42,2L,VectorBase,CDS,208897,209394,NaN,+,0.0,AGAP004679-PB,AGAP004679-RB,NaN
43,2L,VectorBase,exon,209473,209816,NaN,+,NaN,NaN,AGAP004679-RB,AGAP004679-RB-E5


In [11]:
#this gene has a 5' utr
utrintron5 = "AGAP004679-RB"
df = ag3.snp_effects(utrintron5, site_mask)

transcript : AGAP004679-RB
chromosome : 2L 
start : 207894
stop : 210460
strand : +


In [12]:
df[178:]

,position,ref_allele,alt_alleles,effect,impact,ref_codon,alt_codon,aa_pos,ref_aa,alt_aa,aa_change
178,207953,T,C,FIVE_PRIME_UTR,LOW,None,None,NaN,None,None,None
179,207953,T,G,FIVE_PRIME_UTR,LOW,None,None,NaN,None,None,None
180,207954,G,A,INTRAGENIC,LOW,None,None,NaN,None,None,None
181,207954,G,C,INTRAGENIC,LOW,None,None,NaN,None,None,None
182,207954,G,T,INTRAGENIC,LOW,None,None,NaN,None,None,None
...,...,...,...,...,...,...,...,...,...,...,...
7681,210459,A,T,THREE_PRIME_UTR,LOW,None,None,NaN,None,None,None
7682,210459,A,G,THREE_PRIME_UTR,LOW,None,None,NaN,None,None,None
7683,210460,G,A,THREE_PRIME_UTR,LOW,None,None,NaN,None,None,None
7684,210460,G,C,THREE_PRIME_UTR,LOW,None,None,NaN,None,None,None


In [ ]:
df[1490:]

### introns in 3' UTRs

In [ ]:
AGAP028431-RA

In [29]:
#this gene has a 3' utr
utrintron3 = "AGAP029346-RA"
site_mask = 'gamb_colu'
df = ag3.snp_effects(utrintron3, site_mask)

transcript : AGAP029346-RA
chromosome : 2L 
start : 5605327
stop : 5606838
strand : -


In [ ]:
three_prime_UTR	5605327	5605863
three_prime_UTR	5605952	5606353

In [58]:
df[1590:]

,position,ref_allele,alt_alleles,effect,impact,ref_codon,alt_codon,aa_pos,ref_aa,alt_aa,aa_change
1590,5605863,T,A,THREE_PRIME_UTR,LOW,None,None,NaN,None,None,None
1591,5605863,T,C,THREE_PRIME_UTR,LOW,None,None,NaN,None,None,None
1592,5605863,T,G,THREE_PRIME_UTR,LOW,None,None,NaN,None,None,None
1593,5605864,C,A,INTRAGENIC,LOW,None,None,NaN,None,None,None
1594,5605864,C,T,INTRAGENIC,LOW,None,None,NaN,None,None,None
...,...,...,...,...,...,...,...,...,...,...,...
4303,5606837,G,C,FIVE_PRIME_UTR,LOW,None,None,NaN,None,None,None
4304,5606837,G,T,FIVE_PRIME_UTR,LOW,None,None,NaN,None,None,None
4305,5606838,A,C,FIVE_PRIME_UTR,LOW,None,None,NaN,None,None,None
4306,5606838,A,T,FIVE_PRIME_UTR,LOW,None,None,NaN,None,None,None


# SNP allele frequencies

In [ ]:
%%time
df_meta, gt = ag3.snp_allele_frequencies("AGAP004707-RD", "gamb_colu")

In [ ]:
df

In [ ]:
len(gt) *3

In [ ]:
df_meta.loc[df_meta.country == 'AG1000G-UG']
